In [1]:
import numpy as np
import os

from utils import *
# from networks import *
from data import *
from configs import config

from torch.utils.data import DataLoader
# from diffusers import AutoencoderTiny

from tqdm import tqdm
print(os.getcwd())

/home/ee/anaconda3/envs/TaN2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/media/ee/DATA/Talha_Nehal/Diffusion-Codes/GMS-testing


In [2]:
path = f'/media/ee/DATA/Talha_Nehal/Diffusion-Codes/GMS/ckpt/tiny_vae_busi/epochs_200/checkpoints/{CHOSEN_STRATEGY}.pth'
mapping_model3 = get_cuda(ResAttnUNet_DS(**MODEL_PARAMS)).to(dtype=torch.float32)

mapping_model3 = load_checkpoint(mapping_model3, path)
mapping_model3.eval()

empty_dict3 = {}
for key, value in mapping_model3.state_dict().items():
    empty_dict3[key] = value

In [3]:
mapping_model = get_cuda(ResAttnUNet_DS(**MODEL_PARAMS)).to(dtype=torch.float32)
# Print state dict keys and value
print("Mapping model state dict keys and their values:")
empty_dict = {}
for key, value in mapping_model.state_dict().items():
    empty_dict[key] = value

Mapping model state dict keys and their values:


In [4]:
mapping_model2 = get_cuda(ResAttnUNet_DS(**MODEL_PARAMS)).to(dtype=torch.float32)

mapping_model2 = load_checkpoint(mapping_model2, MODEL_WEIGHT_PATH)
mapping_model2.eval()

empty_dict2 = {}
for key, value in mapping_model2.state_dict().items():
    empty_dict2[key] = value

In [ ]:
empty_dict['conv1_0.resblock.conv2.0.bias'], empty_dict2['conv1_0.resblock.conv2.0.bias'], empty_dict3['conv1_0.resblock.conv2.0.bias']

In [ ]:
# compare empty_dict and empty_dict2
i = 0
for key in empty_dict:
    if key in empty_dict2:
        if not torch.equal(empty_dict[key], empty_dict2[key]):
            print(f"Mismatch in {key}")
    else:
        print(f"Key {key} not found in empty_dict2")
    i += 1
    if i > 10:
        break

In [ ]:
# Show a nice visualization of a random set of keys and values for both empty_dict and empty_dict2 
import random
keys = list(empty_dict.keys())
random_keys = random.sample(keys, 5)
for key in random_keys:
    print(f"{key}: {empty_dict[key]} vs {empty_dict2[key]}")

In [9]:
import torch, sys, os

def show_stats(ckpt_path: str, n_layers: int = 5):
    if not os.path.isfile(ckpt_path):
        print(f"[!] File not found: {ckpt_path}")
        return

    state_full  = torch.load(ckpt_path, map_location="cpu", weights_only = True)

    # ‼️ Handle both {"model": …} and plain state-dict formats
    state_dict  = state_full["model"] if isinstance(state_full, dict) and "model" in state_full else state_full

    print(f"\n=== {ckpt_path} ===")
    for i, (name, tensor) in enumerate(state_dict.items()):
        if i >= n_layers: break
        print(f"{name:40s} | mean: {tensor.mean():+.4f} | std: {tensor.std():.4f}")

# ---- usage ----
show_stats(MODEL_WEIGHT_PATH)
show_stats(path)



=== ./ckpt/busi/test_code_2/checkpoints/best_valid_dice.pth ===
input_blocks.weight                      | mean: +0.0021 | std: 0.0902
input_blocks.bias                        | mean: -0.0005 | std: 0.0215
conv1_0.resblock.act1.weight             | mean: +0.1107 | std: nan
conv1_0.resblock.act2.weight             | mean: +0.1107 | std: nan
conv1_0.resblock.conv1.0.weight          | mean: +0.9660 | std: 0.0449

=== /media/ee/DATA/Talha_Nehal/Diffusion-Codes/GMS/ckpt/tiny_vae_busi/epochs_200/checkpoints/best_valid_dice.pth ===
input_blocks.weight                      | mean: +0.0042 | std: 0.0921
input_blocks.bias                        | mean: -0.0002 | std: 0.0270
conv1_0.resblock.act1.weight             | mean: +0.1297 | std: nan
conv1_0.resblock.act2.weight             | mean: +0.0421 | std: nan
conv1_0.resblock.conv1.0.weight          | mean: +0.9659 | std: 0.0509


/tmp/ipykernel_2469509/4082065693.py:16: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /opt/conda/conda-bld/pytorch_1729647369228/work/aten/src/ATen/native/ReduceOps.cpp:1823.)
  print(f"{name:40s} | mean: {tensor.mean():+.4f} | std: {tensor.std():.4f}")


In [8]:
for name, param in mapping_model3.named_parameters():
    if torch.isnan(param).any():
        print(f"[NaN WARNING] {name} has NaNs!")